# Material Color Range Validation - Model Comparison

This notebook demonstrates how to build and compare five different AI models to determine whether an image of a material is **in-range** or **out-of-range** based on its color/lightness. We will use transfer learning with a pre-trained convolutional neural network (CNN) backbone and evaluate the following approaches:

- **Multi-Class Classification** – 5-way classification (in-range: light, standard, dark; out-of-range: too light, too dark).
- **Binary Classification** – 2-way classification (in-range vs. out-of-range).
- **Regression** – Predict a continuous brightness score and threshold it to classify range.
- **Ordinal Regression** – Treat the problem as an ordinal classification, exploiting the ordered nature of brightness levels.
- **Hybrid Multi-Task** – A model with two heads: one for 5-class classification and one for regression, sharing a common CNN base.

We will load images from directories corresponding to the five categories, apply preprocessing and augmentation, then train and fine-tune each model. Finally, we evaluate the models on a test set, compare their accuracies, and export the models to TensorFlow Lite for deployment.

> **Note:** We set random seeds for reproducibility and optimize the workflow for performance (e.g., using caching and prefetching for efficient data loading). The code is written modularly so that you can easily adapt it to different materials by changing the dataset path or material name.

In [ ]:
import os, random, pathlib
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Define the material dataset directory (change material_name to reuse for a different material)
material_name = "MediumCherry"
base_data_dir = pathlib.Path("/path/to/dataset") / material_name  # <-- update this path as needed

# Define class names in order of increasing darkness (brightness decreases from first to last)
class_names = ["out_of_range_too_light", "in_range_light", "in_range_standard", "in_range_dark", "out_of_range_too_dark"]

# Verify that the directories exist
for cname in class_names:
    if not (base_data_dir / cname).exists():
        raise FileNotFoundError(f"Directory not found: {base_data_dir/cname} (please check the path and folder names)")

# Collect all image file paths and their class labels
image_paths = []
labels = []
for idx, cname in enumerate(class_names):
    for filepath in (base_data_dir / cname).glob("*.*"):
        image_paths.append(str(filepath))
        labels.append(idx)
image_paths = np.array(image_paths)
labels = np.array(labels)
num_images = len(labels)
print(f"Found {num_images} images for material '{material_name}'.")

# Shuffle and split into train/validation/test (80/10/10 split)
indices = np.arange(num_images)
np.random.shuffle(indices)
train_end = int(0.8 * num_images)
val_end = int(0.9 * num_images)
train_indices = indices[:train_end]
val_indices   = indices[train_end:val_end]
test_indices  = indices[val_end:]
train_paths, train_labels = image_paths[train_indices], labels[train_indices]
val_paths,   val_labels   = image_paths[val_indices],   labels[val_indices]
test_paths,  test_labels  = image_paths[test_indices],  labels[test_indices]
print(f"Split: {len(train_labels)} training, {len(val_labels)} validation, {len(test_labels)} test images.")

# Create TensorFlow Dataset objects from file paths and labels
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
val_ds   = tf.data.Dataset.from_tensor_slices((val_paths, val_labels))
test_ds  = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))

# Function to load and resize an image from a file path
def load_and_resize(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_image(image, channels=3, expand_animations=False)  # decode JPEG/PNG/etc.
    image = tf.image.resize(image, [224, 224])  # resize to 224x224
    image = tf.cast(image, tf.float32)         # convert to float32
    return image, label

# Apply the loading function and cache the data (to RAM) for faster reuse
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.map(load_and_resize, num_parallel_calls=AUTOTUNE).cache()
val_ds   = val_ds.map(load_and_resize, num_parallel_calls=AUTOTUNE).cache()
test_ds  = test_ds.map(load_and_resize, num_parallel_calls=AUTOTUNE).cache()

# Define data augmentation pipeline for training images
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),           # random horizontal flip
    layers.RandomRotation(0.05),                   # random rotation (±5%)
    layers.RandomZoom(0.1),                        # random zoom
    # layers.RandomTranslation(...) could be added for shifts
])
# Note: We avoid color/brightness augmentations, as those would alter the label (brightness range).

# Normalize pixel values from [0,255] to [0,1]
normalization_layer = layers.Rescaling(1./255)

# Apply augmentation (training only) and normalization
train_ds = train_ds.map(lambda img, lbl: (data_augmentation(img, training=True), lbl), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.map(lambda img, lbl: (normalization_layer(img), lbl), num_parallel_calls=AUTOTUNE)
val_ds   = val_ds.map(lambda img, lbl: (normalization_layer(img), lbl), num_parallel_calls=AUTOTUNE)
test_ds  = test_ds.map(lambda img, lbl: (normalization_layer(img), lbl), num_parallel_calls=AUTOTUNE)

# Batch and prefetch the datasets for efficiency
batch_size = 32
train_ds = train_ds.shuffle(buffer_size=1000, seed=42, reshuffle_each_iteration=True)
train_ds = train_ds.batch(batch_size).prefetch(AUTOTUNE)
val_ds   = val_ds.batch(batch_size).prefetch(AUTOTUNE)
test_ds  = test_ds.batch(batch_size).prefetch(AUTOTUNE)

# Define brightness values for each class (for regression target)
# These are hypothetical "brightness scores" for each category, 0=darkest, 1=brightest
brightness_values = tf.constant([0.9, 0.75, 0.5, 0.25, 0.1], dtype=tf.float32)
# Define thresholds for in-range vs out-of-range based on the midpoints between categories
upper_threshold = (float(brightness_values[0]) + float(brightness_values[1])) / 2.0  # between "too_light" and "in_range_light"
lower_threshold = (float(brightness_values[-2]) + float(brightness_values[-1])) / 2.0  # between "in_range_dark" and "too_dark"
print(f"Brightness thresholds for 'in-range': lower={lower_threshold:.3f}, upper={upper_threshold:.3f}")

## 2. Model Development

We use transfer learning with EfficientNetB0 as the CNN base. For each model, we freeze the base initially and add new layers on top appropriate for the task.

### a. Multi-Class Classification Model

In [ ]:
# a. Multi-Class Classification Model
# Base CNN model (pre-trained EfficientNetB0)
base_model_multi = tf.keras.applications.EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
base_model_multi.trainable = False  # freeze base model layers initially

# Add top layers for multi-class classification
inputs = keras.Input(shape=(224, 224, 3))
x = base_model_multi(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)  # dropout for regularization
output_multi = layers.Dense(5, activation="softmax")(x)  # 5 classes softmax

model_multi = keras.Model(inputs, output_multi, name="MultiClassModel")
model_multi.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                    loss="categorical_crossentropy", metrics=["accuracy"])
model_multi.summary()

### b. Binary Classification Model

In [ ]:
# b. Binary Classification Model
base_model_bin = tf.keras.applications.EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
base_model_bin.trainable = False

inputs = keras.Input(shape=(224, 224, 3))
x = base_model_bin(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
output_bin = layers.Dense(1, activation="sigmoid")(x)  # single sigmoid output

model_binary = keras.Model(inputs, output_bin, name="BinaryModel")
model_binary.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                     loss="binary_crossentropy", metrics=["accuracy"])
model_binary.summary()

### c. Regression Model

In [ ]:
# c. Regression Model
base_model_reg = tf.keras.applications.EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
base_model_reg.trainable = False

inputs = keras.Input(shape=(224, 224, 3))
x = base_model_reg(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
output_reg = layers.Dense(1, activation="linear")(x)  # linear output for brightness score

model_regression = keras.Model(inputs, output_reg, name="RegressionModel")
model_regression.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                         loss="mse")
model_regression.summary()

### d. Ordinal Regression Model

In [ ]:
# d. Ordinal Regression Model
base_model_ord = tf.keras.applications.EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
base_model_ord.trainable = False

inputs = keras.Input(shape=(224, 224, 3))
x = base_model_ord(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
output_ord = layers.Dense(4, activation="sigmoid")(x)  # 4 sigmoid outputs for ordinal thresholds

model_ordinal = keras.Model(inputs, output_ord, name="OrdinalModel")
model_ordinal.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                      loss="binary_crossentropy")
model_ordinal.summary()

### e. Hybrid Multi-Task Model

In [ ]:
# e. Hybrid Multi-Task Model
base_model_mt = tf.keras.applications.EfficientNetB0(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
base_model_mt.trainable = False

inputs = keras.Input(shape=(224, 224, 3))
x = base_model_mt(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
## Classification head (5 classes)
class_output = layers.Dense(5, activation="softmax", name="class_output")(x)
## Regression head (brightness)
reg_output = layers.Dense(1, activation="linear", name="reg_output")(x)

model_multi_task = keras.Model(inputs, [class_output, reg_output], name="MultiTaskModel")
model_multi_task.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
                         loss={"class_output": "categorical_crossentropy", "reg_output": "mse"},
                         metrics={"class_output": "accuracy"})
model_multi_task.summary()

## 3. Training and Fine-Tuning

For each model, we first train only the new top layers (with the EfficientNet base frozen). Then we unfreeze the last few layers of the base for fine-tuning using a lower learning rate.

### Multi-Class Model Training & Fine-Tuning

In [ ]:
# Train the Multi-Class Classification model
epochs_initial = 5
epochs_finetune = 5

# One-hot encode the training and validation labels for 5-class classification
train_ds_multi = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=5)))
val_ds_multi = val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=5)))

# Initial training (base frozen)
history_multi = model_multi.fit(train_ds_multi, validation_data=val_ds_multi, epochs=epochs_initial)

# Fine-tuning: unfreeze last few layers of the base model
base_model_multi.trainable = True
# Freeze all layers except the last 20 layers in the base model (as an example)
for layer in base_model_multi.layers[:-20]:
    layer.trainable = False
# Recompile with a lower learning rate for fine-tuning
model_multi.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                    loss="categorical_crossentropy", metrics=["accuracy"])
# Continue training
history_multi_ft = model_multi.fit(train_ds_multi, validation_data=val_ds_multi,
                                   epochs=epochs_initial+epochs_finetune, initial_epoch=history_multi.epoch[-1] + 1)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Combine history from initial and fine-tune phases
acc = history_multi.history['accuracy'] + history_multi_ft.history.get('accuracy', [])
val_acc = history_multi.history['val_accuracy'] + history_multi_ft.history.get('val_accuracy', [])
loss = history_multi.history['loss'] + history_multi_ft.history.get('loss', [])
val_loss = history_multi.history['val_loss'] + history_multi_ft.history.get('val_loss', [])

epochs_range = range(1, len(acc) + 1)
plt.figure(figsize=(8,4))
# Plot loss
plt.subplot(1,2,1)
plt.plot(epochs_range, loss, label='Train Loss')
plt.plot(epochs_range, val_loss, label='Val Loss')
plt.title('Multi-Class Model Loss')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
# Plot accuracy
plt.subplot(1,2,2)
plt.plot(epochs_range, acc, label='Train Accuracy')
plt.plot(epochs_range, val_acc, label='Val Accuracy')
plt.title('Multi-Class Model Accuracy')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend()
plt.show()

### Binary Model Training & Fine-Tuning

In [ ]:
# Train the Binary Classification model
epochs_initial = 5
epochs_finetune = 5

# Prepare binary labels: 1 for in-range (classes 1,2,3), 0 for out-of-range (classes 0,4)
train_ds_bin = train_ds.map(lambda x, y: (x, tf.cast((y >= 1) & (y <= 3), tf.float32)))
val_ds_bin = val_ds.map(lambda x, y: (x, tf.cast((y >= 1) & (y <= 3), tf.float32)))

history_bin = model_binary.fit(train_ds_bin, validation_data=val_ds_bin, epochs=epochs_initial)

# Fine-tune base model (unfreeze some layers)
base_model_bin.trainable = True
for layer in base_model_bin.layers[:-20]:
    layer.trainable = False
model_binary.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                     loss="binary_crossentropy", metrics=["accuracy"])
history_bin_ft = model_binary.fit(train_ds_bin, validation_data=val_ds_bin,
                                  epochs=epochs_initial+epochs_finetune, initial_epoch=history_bin.epoch[-1] + 1)

# Plot training curves for binary model
acc_bin = history_bin.history['accuracy'] + history_bin_ft.history.get('accuracy', [])
val_acc_bin = history_bin.history['val_accuracy'] + history_bin_ft.history.get('val_accuracy', [])
loss_bin = history_bin.history['loss'] + history_bin_ft.history.get('loss', [])
val_loss_bin = history_bin.history['val_loss'] + history_bin_ft.history.get('val_loss', [])
epochs_range_bin = range(1, len(acc_bin) + 1)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(epochs_range_bin, loss_bin, label='Train Loss')
plt.plot(epochs_range_bin, val_loss_bin, label='Val Loss')
plt.title('Binary Model Loss')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
plt.subplot(1,2,2)
plt.plot(epochs_range_bin, acc_bin, label='Train Accuracy')
plt.plot(epochs_range_bin, val_acc_bin, label='Val Accuracy')
plt.title('Binary Model Accuracy')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend()
plt.show()

### Regression Model Training & Fine-Tuning

In [ ]:
# Train the Regression model
epochs_initial = 5
epochs_finetune = 5

# Prepare regression labels (brightness scores)
train_ds_reg = train_ds.map(lambda x, y: (x, tf.gather(brightness_values, tf.cast(y, tf.int32))))
val_ds_reg = val_ds.map(lambda x, y: (x, tf.gather(brightness_values, tf.cast(y, tf.int32))))

history_reg = model_regression.fit(train_ds_reg, validation_data=val_ds_reg, epochs=epochs_initial)

# Fine-tune base model
base_model_reg.trainable = True
for layer in base_model_reg.layers[:-20]:
    layer.trainable = False
model_regression.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), loss="mse")
history_reg_ft = model_regression.fit(train_ds_reg, validation_data=val_ds_reg,
                                      epochs=epochs_initial+epochs_finetune, initial_epoch=history_reg.epoch[-1] + 1)

# Plot training curves for regression model
loss_reg = history_reg.history['loss'] + history_reg_ft.history.get('loss', [])
val_loss_reg = history_reg.history['val_loss'] + history_reg_ft.history.get('val_loss', [])
epochs_range_reg = range(1, len(loss_reg) + 1)

plt.figure(figsize=(6,4))
plt.plot(epochs_range_reg, loss_reg, label='Train MSE')
plt.plot(epochs_range_reg, val_loss_reg, label='Val MSE')
plt.title('Regression Model Training (MSE)')
plt.xlabel('Epoch'); plt.ylabel('Mean Squared Error'); plt.legend()
plt.show()

### Ordinal Model Training & Fine-Tuning

In [ ]:
# Train the Ordinal regression model
epochs_initial = 5
epochs_finetune = 5

# Prepare ordinal labels (4 binary values) from class label
def ordinal_targets(label):
    # label is scalar tensor (0-4), compare with [0,1,2,3]
    return tf.cast(label > tf.range(4, dtype=tf.int32), tf.float32)

train_ds_ord = train_ds.map(lambda x, y: (x, ordinal_targets(y)))
val_ds_ord = val_ds.map(lambda x, y: (x, ordinal_targets(y)))

history_ord = model_ordinal.fit(train_ds_ord, validation_data=val_ds_ord, epochs=epochs_initial)

# Fine-tune base model
base_model_ord.trainable = True
for layer in base_model_ord.layers[:-20]:
    layer.trainable = False
model_ordinal.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), loss="binary_crossentropy")
history_ord_ft = model_ordinal.fit(train_ds_ord, validation_data=val_ds_ord,
                                   epochs=epochs_initial+epochs_finetune, initial_epoch=history_ord.epoch[-1] + 1)

# Plot training curves for ordinal model
loss_ord = history_ord.history['loss'] + history_ord_ft.history.get('loss', [])
val_loss_ord = history_ord.history['val_loss'] + history_ord_ft.history.get('val_loss', [])
epochs_range_ord = range(1, len(loss_ord) + 1)

plt.figure(figsize=(6,4))
plt.plot(epochs_range_ord, loss_ord, label='Train Loss')
plt.plot(epochs_range_ord, val_loss_ord, label='Val Loss')
plt.title('Ordinal Model Training Loss')
plt.xlabel('Epoch'); plt.ylabel('Binary Crossentropy Loss'); plt.legend()
plt.show()

### Multi-Task Model Training & Fine-Tuning

In [ ]:
# Train the Hybrid Multi-Task model
epochs_initial = 5
epochs_finetune = 5

# Prepare multi-task labels: (one-hot class, brightness)
train_ds_mt = train_ds.map(lambda x, y: (x, (tf.one_hot(y, depth=5), tf.gather(brightness_values, tf.cast(y, tf.int32)))))
val_ds_mt = val_ds.map(lambda x, y: (x, (tf.one_hot(y, depth=5), tf.gather(brightness_values, tf.cast(y, tf.int32)))))

history_mt = model_multi_task.fit(train_ds_mt, validation_data=val_ds_mt, epochs=epochs_initial)

# Fine-tune base model
base_model_mt.trainable = True
for layer in base_model_mt.layers[:-20]:
    layer.trainable = False
model_multi_task.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                         loss={"class_output": "categorical_crossentropy", "reg_output": "mse"},
                         metrics={"class_output": "accuracy"})
history_mt_ft = model_multi_task.fit(train_ds_mt, validation_data=val_ds_mt,
                                     epochs=epochs_initial+epochs_finetune, initial_epoch=history_mt.epoch[-1] + 1)

# Plot training curves for multi-task model (classification head performance)
acc_mt = history_mt.history['class_output_accuracy'] + history_mt_ft.history.get('class_output_accuracy', [])
val_acc_mt = history_mt.history['val_class_output_accuracy'] + history_mt_ft.history.get('val_class_output_accuracy', [])
loss_mt = history_mt.history['loss'] + history_mt_ft.history.get('loss', [])
val_loss_mt = history_mt.history['val_loss'] + history_mt_ft.history.get('val_loss', [])
epochs_range_mt = range(1, len(acc_mt) + 1)

plt.figure(figsize=(8,4))
plt.subplot(1,2,1)
plt.plot(epochs_range_mt, loss_mt, label='Train Total Loss')
plt.plot(epochs_range_mt, val_loss_mt, label='Val Total Loss')
plt.title('Multi-Task Model Loss')
plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.legend()
plt.subplot(1,2,2)
plt.plot(epochs_range_mt, acc_mt, label='Train Class Accuracy')
plt.plot(epochs_range_mt, val_acc_mt, label='Val Class Accuracy')
plt.title('Multi-Task Model Classification Accuracy')
plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.legend()
plt.show()

## 4. Evaluation and Comparison

We evaluate each model on the test set. For the multi-class and ordinal models, we convert the predictions to a binary decision (in-range vs. out-of-range). In-range is defined as classes 1, 2, or 3, and out-of-range as classes 0 or 4.

In [ ]:
# Evaluate on test set

# Multi-class model: get predictions and compute binary accuracy
y_pred_multi_proba = model_multi.predict(test_ds_multi)  # probabilities for 5 classes
y_pred_multi_class = np.argmax(y_pred_multi_proba, axis=1)
y_true_multi_class = test_labels  # original labels 0-4
multi_class_accuracy = np.mean(y_pred_multi_class == y_true_multi_class)  # 5-class accuracy

# Convert to in-range vs out-of-range (in-range: classes 1,2,3)
y_pred_multi_inrange = np.isin(y_pred_multi_class, [1,2,3]).astype(int)
y_true_inrange = np.isin(y_true_multi_class, [1,2,3]).astype(int)
multi_binary_accuracy = np.mean(y_pred_multi_inrange == y_true_inrange)

# Binary model: directly evaluate accuracy on test set
loss_bin, binary_accuracy = model_binary.evaluate(test_ds_bin, verbose=0)

# Regression model: predict brightness and threshold for classification
y_pred_reg = model_regression.predict(test_ds_reg).ravel()  # predicted brightness scores
# Classify as in-range (1) if between lower_threshold and upper_threshold, else out-of-range (0)
y_pred_reg_inrange = np.where((y_pred_reg >= lower_threshold) & (y_pred_reg <= upper_threshold), 1, 0)
y_true_inrange = np.isin(test_labels, [1,2,3]).astype(int)  # reuse from above
reg_binary_accuracy = np.mean(y_pred_reg_inrange == y_true_inrange)

# Ordinal model: predict and decode ordinal outputs to classes, then to in-range
y_pred_ord = model_ordinal.predict(test_ds_ord)
y_pred_ord_class = []
for pred in y_pred_ord:
    ord_class = 4  # default to last class
    for k, p in enumerate(pred):
        if p < 0.5:
            ord_class = k
            break
    y_pred_ord_class.append(ord_class)
y_pred_ord_class = np.array(y_pred_ord_class, dtype=int)
ordinal_class_accuracy = np.mean(y_pred_ord_class == y_true_multi_class)  # 5-class accuracy for ordinal
# Convert to binary in-range
y_pred_ord_inrange = np.isin(y_pred_ord_class, [1,2,3]).astype(int)
ord_binary_accuracy = np.mean(y_pred_ord_inrange == y_true_inrange)

# Multi-task model: it outputs both class and reg, evaluate classification output
y_pred_mt_class_proba = model_multi_task.predict(test_ds_mt)[0]  # model.predict returns [class_probs, reg_pred]
y_pred_mt_class = np.argmax(y_pred_mt_class_proba, axis=1)
multi_task_class_accuracy = np.mean(y_pred_mt_class == y_true_multi_class)
y_pred_mt_inrange = np.isin(y_pred_mt_class, [1,2,3]).astype(int)
mt_binary_accuracy = np.mean(y_pred_mt_inrange == y_true_inrange)

print("Test Accuracy (In-Range vs Out-of-Range):")
print(f"Multi-Class Model:      {multi_binary_accuracy*100:.2f}%")
print(f"Binary Classification:  {binary_accuracy*100:.2f}%")
print(f"Regression Model:       {reg_binary_accuracy*100:.2f}%")
print(f"Ordinal Regression:     {ord_binary_accuracy*100:.2f}%")
print(f"Multi-Task Model:       {mt_binary_accuracy*100:.2f}%")

import pandas as pd
acc_data = {
    "Model": ["Multi-Class", "Binary", "Regression", "Ordinal", "Multi-Task"],
    "Test Accuracy (%)": [multi_binary_accuracy*100, binary_accuracy*100, reg_binary_accuracy*100, ord_binary_accuracy*100, mt_binary_accuracy*100]
}
acc_df = pd.DataFrame(acc_data)
display(acc_df)

# Bar chart of accuracies
plt.figure(figsize=(6,4))
plt.bar(acc_data["Model"], acc_data["Test Accuracy (%)"], color=['C0','C1','C2','C3','C4'])
plt.title("Model Accuracy on Test Set (In-Range vs Out-of-Range)")
plt.ylabel("Accuracy (%)")
plt.ylim(0, 100)
for i, v in enumerate(acc_data["Test Accuracy (%)"]):
    plt.text(i, v+1, f"{v:.1f}%", ha='center')
plt.show()

## 5. Model Export and Reusability

Finally, we export the trained models to TensorFlow Lite (TFLite) format so they can be deployed to mobile or edge devices. The saved files are named with the material name and model type (e.g., `MediumCherry_multi_class.tflite`).

To adapt this notebook for a different material, simply update the `material_name` and `base_data_dir` at the top. The rest of the pipeline is modular and will automatically process the new dataset (provided it follows the same folder structure).

In [ ]:
# Export models to TensorFlow Lite
export_dir = "./tflite_models"
os.makedirs(export_dir, exist_ok=True)

models_to_export = {
    "multi_class": model_multi,
    "binary": model_binary,
    "regression": model_regression,
    "ordinal": model_ordinal,
    "multi_task": model_multi_task
}

for name, model in models_to_export.items():
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    tflite_model = converter.convert()
    tflite_path = os.path.join(export_dir, f"{material_name}_{name}.tflite")
    with open(tflite_path, "wb") as f:
        f.write(tflite_model)
    print(f"Saved {name} model to {tflite_path}")